In [1]:
import pandas as pd
import numpy as np
import os
import sys
import json
import requests
import time
import copy
sys.path.insert(0, '/root/sxw/edit')

from LLINBO_agent import (
    ProblemContext, Parameter, PriorExperiment, 
    LLMConfig, LLINBOAgent
)

# 设置环境变量
os.environ.setdefault("OPENAI_API_KEY", "key")
os.environ.setdefault("HTTP_PROXY", "http://127.0.0.1:7890")
os.environ.setdefault("HTTPS_PROXY", "http://127.0.0.1:7890")

print("✅ 库导入完成")

✅ 库导入完成


In [2]:
def load_chemical_data(csv_path: str):
    """加载化学实验数据"""
    df = pd.read_csv(csv_path)
    return df


def build_parameter_space(df: pd.DataFrame):
    """Construct the parameter space definition for the chemical experiment."""

    parameters = [
        Parameter(
            name="base",
            type="choice",
            values=sorted(df["base"].unique().tolist()),
            value_type="str",
            description="Type of base used in the reaction (e.g., CsOAc, KOAc, etc.), which strongly affects the yield and selectivity.",
        ),
        Parameter(
            name="ligand",
            type="choice",
            values=sorted(df["ligand"].unique().tolist()),
            value_type="str",
            description="Type of ligand utilized to stabilize the catalyst; ligands modulate reactivity and product distribution.",
        ),
        Parameter(
            name="solvent",
            type="choice",
            values=sorted(df["solvent"].unique().tolist()),
            value_type="str",
            description="Type of solvent employed during the reaction; chosen for solubility and reaction rate optimization.",
        ),
        Parameter(
            name="concentration",
            type="choice",
            values=sorted(df["concentration"].unique().tolist()),
            value_type="float",
            description="Concentration of reactants in molarity (M); controls the collision rate of molecules and thus the kinetics.",
        ),
        Parameter(
            name="temperature",
            type="choice",
            values=sorted(df["temperature"].unique().tolist()),
            value_type="int",
            description="Reaction temperature in degrees Celsius (°C); higher or lower temperatures can affect reaction speed and outcomes.",
        ),
    ]
    return parameters


def convert_parameters_to_api_format(parameters):
    """将 LLINBO Parameter 转换为 API ParameterSpace 格式"""
    api_params = []
    for param in parameters:
        api_param = {
            "name": param.name,
            "type": param.type,
            "values": param.values
        }
        api_params.append(api_param)
    return api_params


def call_init_api(parameter_space, objectives, batch_size=20, seed=None, api_url="http://localhost:3320"):
    """调用 init 接口进行采样"""
    init_endpoint = f"{api_url}/init"
    
    # 转换参数空间格式
    api_parameter_space = convert_parameters_to_api_format(parameter_space)
    
    # 构建请求数据
    init_request = {
        "parameter_space": api_parameter_space,
        "objectives": list(objectives.keys()),
        "batch": batch_size,
        "seed": seed,
        "sampling_method": "lhs"  # 使用 lhs 采样
    }
    
    print(f"🚀 调用 init 接口，批次大小: {batch_size}")
    
    try:
        response = requests.post(init_endpoint, json=init_request, timeout=30)
        
        if response.status_code == 200:
            result = response.json()
            print(f"✅ Init 接口调用成功")
            print(f"📈 生成参数组合数: {len(result['results'])}")
            print(f"💬 消息: {result['message']}")
            return result
        else:
            print(f"❌ Init 接口调用失败: {response.status_code}")
            print(f"📄 错误信息: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Init 接口调用异常: {str(e)}")
        return None


def simulate_experiment_results(params_list, df, random_seed=None):
    """从真实数据中查找完全匹配的实验结果"""
    if random_seed is not None:
        np.random.seed(random_seed)
    
    results = []
    
    for params in params_list:
        # 构建精确匹配的查询条件
        query_conditions = []
        for key, value in params.items():
            if isinstance(value, str):
                query_conditions.append(f"{key} == '{value}'")
            else:
                query_conditions.append(f"{key} == {value}")
        
        # 构建查询字符串
        query_str = " and ".join(query_conditions)
        
        try:
            # 查找完全匹配的数据
            matched_data = df.query(query_str)
            
            if len(matched_data) > 0:
                # 取第一个匹配的结果
                row = matched_data.iloc[0]
                experiment_result = {
                    "parameters": params,
                    "metrics": {
                        "yield": float(row['yield']),
                        "cost": float(row['cost'])
                    }
                }
                results.append(experiment_result)
                print(f"✅ 找到完全匹配: {params} -> yield={row['yield']:.2f}, cost={row['cost']:.3f}")
            else:
                print(f"⚠️ 未找到完全匹配: {params}")
                # 如果找不到，使用随机结果
                random_row = df.sample(1, random_state=random_seed).iloc[0]
                experiment_result = {
                    "parameters": params,
                    "metrics": {
                        "yield": float(random_row['yield']),
                        "cost": float(random_row['cost'])
                    }
                }
                results.append(experiment_result)
                print(f"⚠️ 使用随机结果: {params} -> yield={random_row['yield']:.2f}, cost={random_row['cost']:.3f}")
                
        except Exception as e:
            print(f"❌ 查询失败: {str(e)}")
            # 使用随机结果
            random_row = df.sample(1, random_state=random_seed).iloc[0]
            experiment_result = {
                "parameters": params,
                "metrics": {
                    "yield": float(random_row['yield']),
                    "cost": float(random_row['cost'])
                }
            }
            results.append(experiment_result)
    
    return results


def convert_experiment_results_to_prior_experiments(experiment_results):
    """将实验结果转换为 PriorExperiment 格式"""
    prior_experiments = []
    for result in experiment_results:
        exp = PriorExperiment(
            parameters=result["parameters"],
            metrics=result["metrics"]
        )
        prior_experiments.append(exp)
    return prior_experiments

def create_prior_experiments(df: pd.DataFrame, parameters, objectives, problem_context,
                             n_samples: int = 50, 
                             llm_config: LLMConfig = None,
                             seed: int = None):
    """使用 LLINBO Agent 的初始采样方法创建先验实验数据"""
    print(f"\n📚 使用 LLINBO Agent 进行初始采样，生成 {n_samples} 个先验实验数据...")
    
    # 如果没有提供 llm_config，创建一个默认的
    if llm_config is None:
        llm_config = LLMConfig(
            model_name="intern-latest",
            api_key="sk-WP1bzhp1afmGR16ci8XkA7igyeTSUQedYI9P7YHDQRdCbe4m",
            base_url="https://chat.intern-ai.org.cn/api/v1/"
        )
    
    # 创建 LLINBO Agent（没有先验数据，用于初始采样）
    agent = LLINBOAgent(
        problem_context=problem_context,
        parameters=parameters,
        objectives=objectives,
        llm_config=llm_config,
        prior_experiments=None,  # 没有先验数据
        random_seed=seed
    )
    
    # 使用初始采样方法生成参数建议
    print(f"🤖 使用 LLINBO Agent 生成 {n_samples} 个初始采样建议...")
    suggestions = agent.suggest_initial_parameters(
        num_suggestions=n_samples,
        print_prompt=True,
        print_response=True
    )
    
    if not suggestions:
        print("⚠️ LLINBO Agent 未能生成有效建议，使用随机采样作为备选方案")
        # 备选方案：随机采样
        sample_df = df.sample(n=min(n_samples, len(df)), random_state=seed)
        prior_experiments = []
        for _, row in sample_df.iterrows():
            exp = PriorExperiment(
                parameters={
                    "base": str(row["base"]),
                    "ligand": str(row["ligand"]),
                    "solvent": str(row["solvent"]),
                    "concentration": float(row["concentration"]),
                    "temperature": int(row["temperature"]),
                },
                metrics={
                    "yield": float(row["yield"]),
                    "cost": float(row["cost"]),
                }
            )
            prior_experiments.append(exp)
        return prior_experiments
    
    # 从数据集中查找匹配的实验结果
    print(f"\n🔍 从数据集中查找匹配的实验结果...")
    experiment_results = simulate_experiment_results(suggestions, df, random_seed=seed)
    
    # 转换为 PriorExperiment 格式
    prior_experiments = convert_experiment_results_to_prior_experiments(experiment_results)
    
    print(f"✅ 成功创建 {len(prior_experiments)} 个先验实验数据")
    
    return prior_experiments

print("✅ 辅助函数定义完成")

✅ 辅助函数定义完成


In [3]:
csv_path = "test/1728_BMS_experiments_yield_and_cost.csv"
print(f"📊 加载数据集: {csv_path}")
df = load_chemical_data(csv_path)
print(f"✅ 数据集加载成功: {len(df)} 个实验数据")

📊 加载数据集: test/1728_BMS_experiments_yield_and_cost.csv
✅ 数据集加载成功: 1728 个实验数据


In [4]:
problem_context = ProblemContext(
    problem_description=(
        "Optimization of the reaction conditions for a Pd-catalyzed direct C–H "
        "arylation between N1-methyl-1H-imidazole-4-carbonitrile and "
        "1-bromo-2-fluorobenzene. This transformation forms a C(sp2)–C(sp2) "
        "bond selectively at the C-5 position of the imidazole ring through a "
        "Pd(0)/Pd(II) catalytic cycle involving oxidative addition of the aryl "
        "bromide, base-assisted C–H activation (CMD pathway), and reductive "
        "elimination to furnish the heterobiaryl product."
    ),

    industry="Chemical synthesis - organic reaction optimization",

    domain_knowledge="""
    This is an optimization problem for a Pd-catalyzed direct C–H arylation
    reaction, specifically the coupling of N1-methyl-1H-imidazole-4-carbonitrile
    with 1-bromo-2-fluorobenzene. The overall transformation can be represented
    as:

        Im–H  +  Ar–Br  →  Ar–Im  +  HBr (neutralized by base)

    Mechanistic background:
    - The reaction proceeds through a Pd(0)/Pd(II) catalytic cycle:
        (1) Oxidative addition of the aryl bromide to Pd(0)
        (2) Base-assisted C–H activation of the imidazole (CMD-type mechanism)
        (3) Reductive elimination to form the C–C coupled product
        (4) Regeneration of Pd(0)

    - The C-5 position of the imidazole is the most reactive site due to the
      electron-withdrawing nitrile at C-4 and the N-methyl substituent, which
      together influence the electronic distribution and regioselectivity.

    Key domain factors affecting reaction performance:
    - Base influences C–H activation efficiency, acidity of the imidazole C–H
      bond, and HBr neutralization.
    - Ligand strongly modulates oxidative addition rate, reductive elimination,
      catalyst stability, and off-cycle Pd species.
    - Solvent affects catalyst solvation, substrate and base solubility, and the
      prevalence of side reactions (e.g., protodehalogenation or homocoupling).
    - Concentration impacts effective collision frequency, catalyst aggregation,
      and selectivity between productive and unproductive pathways.
    - Temperature influences reaction kinetics, catalyst longevity, and
      the balance between desired arylation and decomposition pathways.

    Typical failure modes in this reaction class:
    - Protodehalogenation of the aryl bromide
    - Homocoupling (biaryl formation)
    - Catalyst deactivation (Pd black)
    - Over-arylation of the heteroarene under harsh conditions

    The optimization objective is to maximize the yield of the mono-arylated
    product while minimizing cost, considering catalyst stability, reagent
    efficiency, and suppression of side reactions.
    """,

    constraints=[
        "All recommended parameter combinations must conform to basic chemical principles."
    ],

    optimization_goals=[
        "Maximize yield",
        "Minimize cost"
    ]
)

In [5]:
print("\n🔧 构建参数空间...")
parameters = build_parameter_space(df)
print(f"✅ 参数空间定义完成: {len(parameters)} 个参数")
for param in parameters:
    print(f"   - {param.name}: {len(param.values)} 个可选值")


# ## 4. 定义优化目标

# %%
objectives = {
    "yield": {"minimize": False},  # 最大化产率
    "cost": {"minimize": True}      # 最小化成本
}
print("✅ 优化目标定义完成")
print(f"   - yield: 最大化")
print(f"   - cost: 最小化")


🔧 构建参数空间...
✅ 参数空间定义完成: 5 个参数
   - base: 4 个可选值
   - ligand: 12 个可选值
   - solvent: 4 个可选值
   - concentration: 3 个可选值
   - temperature: 3 个可选值
✅ 优化目标定义完成
   - yield: 最大化
   - cost: 最小化


In [6]:
print("\n📚 创建先验实验数据...")
prior_experiments = create_prior_experiments(
    df,parameters,objectives,problem_context,5,


)
print(f"✅ 先验实验数据: {len(prior_experiments)} 个实验")


📚 创建先验实验数据...

📚 使用 LLINBO Agent 进行初始采样，生成 5 个先验实验数据...
🤖 使用 LLINBO Agent 生成 5 个初始采样建议...

📝 输入大模型的完整提示词（初始采样模式）:
# Optimization Problem Context
**Problem Description**: Optimization of the reaction conditions for a Pd-catalyzed direct C–H arylation between N1-methyl-1H-imidazole-4-carbonitrile and 1-bromo-2-fluorobenzene. This transformation forms a C(sp2)–C(sp2) bond selectively at the C-5 position of the imidazole ring through a Pd(0)/Pd(II) catalytic cycle involving oxidative addition of the aryl bromide, base-assisted C–H activation (CMD pathway), and reductive elimination to furnish the heterobiaryl product.
**Industry Domain**: Chemical synthesis - organic reaction optimization
**Domain Knowledge**: 
    This is an optimization problem for a Pd-catalyzed direct C–H arylation
    reaction, specifically the coupling of N1-methyl-1H-imidazole-4-carbonitrile
    with 1-bromo-2-fluorobenzene. The overall transformation can be represented
    as:

        Im–H  +  Ar–Br  →  Ar–Im  + 

In [7]:
print("\n🤖 初始化 LLINBO Agent...")
llm_config = LLMConfig(
    model_name="intern-latest",
    api_key="sk-WP1bzhp1afmGR16ci8XkA7igyeTSUQedYI9P7YHDQRdCbe4m",
    base_url="https://chat.intern-ai.org.cn/api/v1/"
)

agent = LLINBOAgent(
    problem_context=problem_context,
    parameters=parameters,
    objectives=objectives,
    llm_config=llm_config,
    prior_experiments=prior_experiments
)
print("✅ LLINBO Agent 初始化完成")


🤖 初始化 LLINBO Agent...
✅ LLINBO Agent 初始化完成


In [7]:
print("\n🎯 生成优化建议...")
num_suggestions = 5
suggestions = agent.suggest_parameters(num_suggestions=num_suggestions, print_prompt=True, print_response=True)
print(f"✅ 成功生成 {len(suggestions)} 个建议")


🎯 生成优化建议...


NameError: name 'agent' is not defined

In [9]:
round_results = simulate_experiment_results(suggestions, df)
round_prior_experiments = convert_experiment_results_to_prior_experiments(round_results)
# 去掉重复的实验（按参数去重）
all_experiments = prior_experiments + round_prior_experiments
# 按参数唯一性去重（假设 parameters 为 dict，可序列化为 tuple）
def unique_experiments(experiments):
    seen = set()
    result = []
    for exp in experiments:
        # 只用 parameters 字典做唯一性判断
        param_tuple = tuple(sorted(exp.parameters.items()))
        if param_tuple not in seen:
            seen.add(param_tuple)
            result.append(exp)
    return result

prior_experiments[:] = unique_experiments(all_experiments)

✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'X-Phos', 'solvent': 'DMAc', 'concentration': 0.153, 'temperature': 105} -> yield=84.48, cost=0.076
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'X-Phos', 'solvent': 'DMAc', 'concentration': 0.1, 'temperature': 120} -> yield=84.81, cost=0.080
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'JackiePhos', 'solvent': 'DMAc', 'concentration': 0.1, 'temperature': 105} -> yield=41.00, cost=0.402
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'X-Phos', 'solvent': 'BuCN', 'concentration': 0.1, 'temperature': 105} -> yield=83.83, cost=0.098
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'X-Phos', 'solvent': 'DMAc', 'concentration': 0.057, 'temperature': 90} -> yield=79.33, cost=0.090


In [10]:

agent = LLINBOAgent(
    problem_context=problem_context,
    parameters=parameters,
    objectives=objectives,
    llm_config=llm_config,
    prior_experiments=prior_experiments
)
print("\n🎯 生成优化建议...")
num_suggestions = 5
suggestions = agent.suggest_parameters(num_suggestions=num_suggestions, print_prompt=True, print_response=True)
print(f"✅ 成功生成 {len(suggestions)} 个建议")


🎯 生成优化建议...

📝 输入大模型的完整提示词:
# Optimization Problem Context
**Problem Description**: Optimization of the reaction conditions for a Pd-catalyzed direct C–H arylation between N1-methyl-1H-imidazole-4-carbonitrile and 1-bromo-2-fluorobenzene. This transformation forms a C(sp2)–C(sp2) bond selectively at the C-5 position of the imidazole ring through a Pd(0)/Pd(II) catalytic cycle involving oxidative addition of the aryl bromide, base-assisted C–H activation (CMD pathway), and reductive elimination to furnish the heterobiaryl product.
**Industry Domain**: Chemical synthesis - organic reaction optimization
**Domain Knowledge**: 
    This is an optimization problem for a Pd-catalyzed direct C–H arylation
    reaction, specifically the coupling of N1-methyl-1H-imidazole-4-carbonitrile
    with 1-bromo-2-fluorobenzene. The overall transformation can be represented
    as:

        Im–H  +  Ar–Br  →  Ar–Im  +  HBr (neutralized by base)

    Mechanistic background:
    - The reaction proceeds th

In [11]:
round_results = simulate_experiment_results(suggestions, df)
round_prior_experiments = convert_experiment_results_to_prior_experiments(round_results)
prior_experiments.extend(round_prior_experiments)

In [12]:
agent = LLINBOAgent(
    problem_context=problem_context,
    parameters=parameters,
    objectives=objectives,
    llm_config=llm_config,
    prior_experiments=prior_experiments
)
print("\n🎯 生成优化建议...")
num_suggestions = 5
suggestions = agent.suggest_parameters(num_suggestions=num_suggestions, print_prompt=True, print_response=True)
print(f"✅ 成功生成 {len(suggestions)} 个建议")


🎯 生成优化建议...

📝 输入大模型的完整提示词:
# Optimization Problem Context
**Problem Description**: Optimization of the reaction conditions for a Pd-catalyzed direct C–H arylation between N1-methyl-1H-imidazole-4-carbonitrile and 1-bromo-2-fluorobenzene. This transformation forms a C(sp2)–C(sp2) bond selectively at the C-5 position of the imidazole ring through a Pd(0)/Pd(II) catalytic cycle involving oxidative addition of the aryl bromide, base-assisted C–H activation (CMD pathway), and reductive elimination to furnish the heterobiaryl product.
**Industry Domain**: Chemical synthesis - organic reaction optimization
**Domain Knowledge**: 
    This is an optimization problem for a Pd-catalyzed direct C–H arylation
    reaction, specifically the coupling of N1-methyl-1H-imidazole-4-carbonitrile
    with 1-bromo-2-fluorobenzene. The overall transformation can be represented
    as:

        Im–H  +  Ar–Br  →  Ar–Im  +  HBr (neutralized by base)

    Mechanistic background:
    - The reaction proceeds th

In [13]:
round_results = simulate_experiment_results(suggestions, df)
round_prior_experiments = convert_experiment_results_to_prior_experiments(round_results)
prior_experiments.extend(round_prior_experiments)
prior_experiments[:] = unique_experiments(prior_experiments)
experiments_df = pd.DataFrame([{
    **exp.parameters,
    **exp.metrics

} for exp in prior_experiments ])
experiments_df

✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'CgMe-PPh', 'solvent': 'DMAc', 'concentration': 0.1, 'temperature': 105} -> yield=83.25, cost=0.104
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'GorlosPhos HBF4', 'solvent': 'DMAc', 'concentration': 0.1, 'temperature': 105} -> yield=19.97, cost=0.084
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'tBPh-CPhos', 'solvent': 'DMAc', 'concentration': 0.1, 'temperature': 105} -> yield=29.31, cost=0.314
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'P(fur)3', 'solvent': 'DMAc', 'concentration': 0.1, 'temperature': 105} -> yield=45.21, cost=0.084
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'PCy3 HBF4', 'solvent': 'DMAc', 'concentration': 0.1, 'temperature': 105} -> yield=12.83, cost=0.078


,base,ligand,solvent,concentration,temperature,yield,cost
0,CsOAc,BrettPhos,DMAc,0.100,105,4.74,0.191274
1,CsOAc,X-Phos,DMAc,0.100,105,87.83,0.080243
2,CsOPiv,BrettPhos,DMAc,0.100,105,0.00,0.232133
3,KOAc,BrettPhos,DMAc,0.100,105,5.47,0.145775
4,CsOAc,BrettPhos,BuCN,0.100,105,64.48,0.209578
5,CsOAc,X-Phos,DMAc,0.153,105,84.48,0.075583
6,CsOAc,X-Phos,DMAc,0.100,120,84.81,0.080234
7,CsOAc,JackiePhos,DMAc,0.100,105,41.00,0.401630
8,CsOAc,X-Phos,BuCN,0.100,105,83.83,0.098456
9,CsOAc,X-Phos,DMAc,0.057,90,79.33,0.090346


In [37]:
experiments_df.to_csv("all_llm_prior_experiments.csv", index=False)

In [ ]:
# ## 初始化多轮测试参数

# %%
# 重新加载数据（如果需要）
csv_path = "test/1728_BMS_experiments_yield_and_cost.csv"
df_multi = load_chemical_data(csv_path)
parameters_multi = build_parameter_space(df_multi)
objectives_multi = {
    "yield": {"minimize": False},  # 最大化产率
    "cost": {"minimize": True}      # 最小化成本
}

problem_context_multi = ProblemContext(
    problem_description="优化 Pd 催化的 C-H 芳基化反应条件，反应物为 N1-甲基-1H-咪唑-4-甲腈和 1-溴-2-氟苯",
    industry="化学合成 - 有机反应优化",
    domain_knowledge="""
    这是一个 Pd 催化的 C-H 芳基化反应优化问题。
    - 碱基（base）影响反应活性和选择性
    - 配体（ligand）对催化剂活性和选择性至关重要
    - 溶剂（solvent）影响反应速率和产物选择性
    - 浓度（concentration）影响反应速率和副反应
    - 温度（temperature）影响反应速率和选择性
    优化目标是同时最大化产率（yield）和最小化成本（cost）。
    """,
    constraints=[
            "所有推荐的参数组合必须符合基本化学常识"

    ],
    optimization_goals=[
        "最大化产率（yield）",
        "最小化成本（cost）"
    ]
)

# 多轮测试配置
n_rounds = 20
init_batch = 10
update_batch = 3
api_url = "http://localhost:3320"
seed = 42

print("✅ 多轮测试参数初始化完成")
print(f"   总轮次: {n_rounds}")
print(f"   初始批次: {init_batch}")
print(f"   更新批次: {update_batch}")

# %% [markdown]
# ## 定义多轮优化函数

# %%

✅ 多轮测试参数初始化完成
   总轮次: 20
   初始批次: 10
   更新批次: 3


In [ ]:
def run_llinbo_optimization(df, parameters, objectives, problem_context, 
                           n_rounds=10, init_batch=10, update_batch=3, 
                           api_url="http://localhost:3320", seed=42):
    """使用 LLINBO Agent 进行多轮优化，仿照 notebook 的方式"""
    
    print(f"\n🔄 开始 {n_rounds} 轮优化循环（使用 LLINBO Agent）")
    print(f"📊 初始批次: {init_batch}, 更新批次: {update_batch}")
    
    all_experiments = []
    optimization_history = []
    
    # 第一轮：初始化（使用 init 接口）
    print(f"\n=== 第 1 轮：初始化 ===")
    init_result = call_init_api(parameters, objectives, batch_size=init_batch, seed=seed, api_url=api_url)
    
    if not init_result:
        print("❌ 初始化失败，退出优化循环")
        return None, None
    
    # 模拟第一轮实验结果
    round_results = simulate_experiment_results(init_result['results'], df, random_seed=seed)
    round_prior_experiments = convert_experiment_results_to_prior_experiments(round_results)
    all_experiments.extend(round_prior_experiments)
    
    # 记录历史
    optimization_history.append({
        'round': 1,
        'type': 'init',
        'parameters': init_result['results'],
        'results': round_results,
        'best_yield': max([r['metrics']['yield'] for r in round_results]),
        'best_cost': min([r['metrics']['cost'] for r in round_results])
    })
    
    print(f"📈 第 1 轮最佳结果: yield={optimization_history[-1]['best_yield']:.2f}, cost={optimization_history[-1]['best_cost']:.3f}")
    
    # 后续轮次：使用 LLINBO Agent（代替 update 接口）
    for round_num in range(2, n_rounds + 1):
        print(f"\n=== 第 {round_num} 轮：LLINBO 优化 ===")
        
        # 重要：在创建 agent 之前，确认上一轮的结果已经加入
        # 显示当前先验数据数量（确认数据已更新）
        print(f"📊 当前先验实验数据数量: {len(all_experiments)} 个")
        if len(all_experiments) > 0:
            # 显示最近几轮的数据，确认包含上一轮的结果
            recent_count = min(5, len(all_experiments))
            recent_yields = [exp.metrics["yield"] for exp in all_experiments[-recent_count:]]
            recent_costs = [exp.metrics["cost"] for exp in all_experiments[-recent_count:]]
            print(f"   最近 {recent_count} 个实验的产率: {[f'{y:.2f}' for y in recent_yields]}")
            print(f"   最近 {recent_count} 个实验的成本: {[f'{c:.4f}' for c in recent_costs]}")
            
            # 显示上一轮（第 round_num-1 轮）的结果是否在当前先验数据中
            if round_num > 2:
                prev_round_history = [h for h in optimization_history if h['round'] == round_num - 1]
                if prev_round_history:
                    prev_round_params = prev_round_history[0]['parameters']
                    print(f"   🔍 检查上一轮（第 {round_num-1} 轮）的结果是否在先验数据中:")
                    for i, prev_param in enumerate(prev_round_params, 1):
                        # 检查这个参数组合是否在 all_experiments 中
                        found = False
                        for exp in all_experiments:
                            if all(exp.parameters.get(k) == prev_param.get(k) for k in prev_param.keys()):
                                found = True
                                print(f"      上一轮建议 {i}: ✅ 已在先验数据中 (yield={exp.metrics['yield']:.2f}, cost={exp.metrics['cost']:.4f})")
                                break
                        if not found:
                            print(f"      上一轮建议 {i}: ❌ 未在先验数据中！")
        
        # 创建 LLINBO Agent（使用当前所有实验数据作为先验）
        # 重要：确保使用更新后的 all_experiments（包含之前所有轮次的结果）
        # 使用深拷贝确保数据独立
        prior_experiments_for_agent = copy.deepcopy(all_experiments)
        
        print(f"🔧 创建 LLINBO Agent，使用 {len(prior_experiments_for_agent)} 个先验实验数据")
        
        llm_config = LLMConfig(
            model_name="gpt-5-mini",
            api_key=os.getenv("OPENAI_API_KEY"),
            base_url="https://api.openai.com/v1"
        )
        
        # 创建新的 agent，传入更新后的先验数据
        agent = LLINBOAgent(
            problem_context=problem_context,
            parameters=parameters,
            objectives=objectives,
            llm_config=llm_config,
            prior_experiments=prior_experiments_for_agent  # 使用深拷贝，确保数据正确传递
        )
        
        # 验证 agent 中的先验数据数量
        print(f"✅ Agent 创建完成，Agent 中的先验数据数量: {len(agent.prior_experiments)} 个")
        
        # 生成优化建议（LLINBO Agent 代替 update 接口）
        print(f"🤖 使用 LLINBO Agent 生成 {update_batch} 个优化建议...")
        
        # 添加延迟，确保不会因为请求过快导致问题
        time.sleep(0.5)  # 在请求前等待，避免请求过快
        
        suggestions = agent.suggest_parameters(num_suggestions=update_batch)
        
        if not suggestions:
            print(f"⚠️ 第 {round_num} 轮未能生成有效建议，跳过")
            continue
        
        print(f"✅ 成功生成 {len(suggestions)} 个建议")
        
        # 模拟实验结果
        round_results = simulate_experiment_results(suggestions, df, random_seed=seed)
        round_prior_experiments = convert_experiment_results_to_prior_experiments(round_results)
        
        # 重要：将本轮结果加入到先验数据中，供下一轮使用
        # 在加入之前，先检查是否有重复
        print(f"📝 准备将第 {round_num} 轮的 {len(round_prior_experiments)} 个实验结果加入到先验数据中")
        for i, new_exp in enumerate(round_prior_experiments, 1):
            # 检查是否已经存在相同的参数组合
            is_duplicate = False
            for existing_exp in all_experiments:
                if all(existing_exp.parameters.get(k) == new_exp.parameters.get(k) 
                       for k in new_exp.parameters.keys()):
                    is_duplicate = True
                    print(f"   ⚠️ 第 {round_num} 轮建议 {i} 的参数组合已存在于先验数据中，跳过重复添加")
                    break
            if not is_duplicate:
                all_experiments.append(new_exp)
                print(f"   ✅ 第 {round_num} 轮建议 {i} 已加入先验数据")
        
        print(f"📊 更新后的先验数据总数: {len(all_experiments)} 个")
        
        # 记录历史
        optimization_history.append({
            'round': round_num,
            'type': 'llinbo',
            'parameters': suggestions,
            'results': round_results,
            'best_yield': max([r['metrics']['yield'] for r in round_results]),
            'best_cost': min([r['metrics']['cost'] for r in round_results])
        })
        
        print(f"📈 第 {round_num} 轮最佳结果: yield={optimization_history[-1]['best_yield']:.2f}, cost={optimization_history[-1]['best_cost']:.3f}")
        

        
        # 添加延迟，避免请求过快
        time.sleep(1)
    
    return all_experiments, optimization_history

print("✅ 多轮优化函数定义完成")

✅ 多轮优化函数定义完成


In [ ]:
llinbo_experiments, llinbo_history = run_llinbo_optimization(
    df_multi, parameters_multi, objectives_multi, problem_context_multi,
    n_rounds=n_rounds,
    init_batch=init_batch,
    update_batch=update_batch,
    api_url=api_url,
    seed=seed
)


🔄 开始 20 轮优化循环（使用 LLINBO Agent）
📊 初始批次: 10, 更新批次: 3

=== 第 1 轮：初始化 ===
🚀 调用 init 接口，批次大小: 10
✅ Init 接口调用成功
📈 生成参数组合数: 10
💬 消息: 初始化成功，使用lhs采样生成10个参数组合
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'PPh3', 'solvent': 'BuCN', 'concentration': 0.1, 'temperature': 90} -> yield=5.80, cost=0.090
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'BrettPhos', 'solvent': 'BuCN', 'concentration': 0.153, 'temperature': 90} -> yield=28.02, cost=0.199
✅ 找到完全匹配: {'base': 'CsOPiv', 'ligand': 'GorlosPhos HBF4', 'solvent': 'p-Xylene', 'concentration': 0.153, 'temperature': 105} -> yield=4.53, cost=0.122
✅ 找到完全匹配: {'base': 'KOAc', 'ligand': 'X-Phos', 'solvent': 'DMAc', 'concentration': 0.057, 'temperature': 120} -> yield=77.58, cost=0.045
✅ 找到完全匹配: {'base': 'KOPiv', 'ligand': 'P(fur)3', 'solvent': 'BuCN', 'concentration': 0.153, 'temperature': 120} -> yield=53.11, cost=0.045
✅ 找到完全匹配: {'base': 'CsOAc', 'ligand': 'PCy3 HBF4', 'solvent': 'BuOAc', 'concentration': 0.057, 'temperature': 105} -> yield=0.00, cost=0.108
✅ 找到完全匹配: 

In [ ]:
import pandas as pd
# 设置 pandas 显示选项，显示所有行和列
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.max_rows', None)     # 显示所有行
pd.set_option('display.width', None)        # 不限制显示宽度
pd.set_option('display.max_colwidth', None) # 不限制列宽度


experiments_df = pd.DataFrame([{
    **exp.parameters,
    **exp.metrics

} for exp in llinbo_experiments])
experiments_df

,base,ligand,solvent,concentration,temperature,yield,cost
0,CsOAc,PPh3,BuCN,0.100,90,5.80,0.089659
1,CsOAc,BrettPhos,BuCN,0.153,90,28.02,0.198802
2,CsOPiv,GorlosPhos HBF4,p-Xylene,0.153,105,4.53,0.121718
3,KOAc,X-Phos,DMAc,0.057,120,77.58,0.044840
4,KOPiv,P(fur)3,BuCN,0.153,120,53.11,0.045232
5,CsOAc,PCy3 HBF4,BuOAc,0.057,105,0.00,0.107935
6,CsOPiv,CgMe-PPh,BuOAc,0.100,105,24.00,0.155989
7,KOPiv,tBPh-CPhos,p-Xylene,0.153,90,24.02,0.264828
8,KOPiv,PPh2Me,DMAc,0.057,90,5.02,0.036398
9,KOAc,PPhtBu2,DMAc,0.057,120,2.13,0.053335
